In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

class ObjectTracker:
    def __init__(self, maxlen=30):  # Track last 30 frames
        self.object_history = {}
        self.maxlen = maxlen

    def update(self, class_name, center, distance):
        if class_name not in self.object_history:
            self.object_history[class_name] = deque(maxlen=self.maxlen)
        self.object_history[class_name].append((center, distance))

    def get_motion_analysis(self, class_name):
        if class_name not in self.object_history or len(self.object_history[class_name]) < 2:
            return "Stationary", 0
        
        positions = self.object_history[class_name]
        last_pos = positions[-1][0]
        prev_pos = positions[-2][0]
        
        dx = last_pos[0] - prev_pos[0]
        dy = last_pos[1] - prev_pos[1]
        speed = np.sqrt(dx**2 + dy**2)
        
        if speed < 5:
            return "Stationary", speed
        elif abs(dx) > abs(dy):
            return "Moving Horizontally", speed
        else:
            return "Moving Vertically", speed

# List of household objects
HOUSEHOLD_OBJECTS = [
    'chair', 'couch', 'bed', 'dining table', 'toilet', 'tv', 
    'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'refrigerator', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'bottle'
]

def calculate_distance(focal_length, known_width, pixel_width):
    return (known_width * focal_length) / pixel_width

def predict_collision(current_pos, history, frame_shape, threshold=100):
    if len(history) < 2:
        return "Safe"
        
    # Calculate trajectory
    current = np.array(current_pos)
    prev = np.array(history[-2][0])
    velocity = current - prev
    
    # Project future position
    future_pos = current + velocity * 10  # Project 10 frames ahead
    
    # Check if future position is too close to camera
    if future_pos[0] < threshold or future_pos[0] > frame_shape[1] - threshold:
        return "Warning: Horizontal Collision"
    if future_pos[1] < threshold or future_pos[1] > frame_shape[0] - threshold:
        return "Warning: Vertical Collision"
        
    return "Safe"

def analyze_stability(distance_history):
    if len(distance_history) < 5:
        return "Insufficient Data"
    
    distances = [d for _, d in distance_history]
    variance = np.var(distances)
    if variance < 2:
        return "Stable"
    elif variance < 5:
        return "Slight Movement"
    else:
        return "Unstable"

# Load YOLO model
model = YOLO('yolov8n.pt')

# Initialize camera
cap = cv2.VideoCapture(0)

# Calibration parameters
KNOWN_DISTANCES = {
    'chair': 30.0,      # inches
    'laptop': 16.0,     # inches
    'bottle': 4.0,      # inches
    'cell phone': 6.0   # inches
}

FOCAL_LENGTHS = {}
tracker = ObjectTracker()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 inference
    results = model(frame)
    
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Get class name and check if it's a household object
            class_name = result.names[int(box.cls[0])]
            if class_name not in HOUSEHOLD_OBJECTS:
                continue
                
            # Get box coordinates and confidence
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = float(box.conf[0])
            
            if confidence < 0.5:  # Filter low confidence detections
                continue
                
            # Calculate width in pixels
            pixel_width = x2 - x1
            
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Calculate and display distance if possible
            if class_name in KNOWN_DISTANCES:
                known_width = KNOWN_DISTANCES[class_name]
                
                # Calibrate focal length if not done
                if class_name not in FOCAL_LENGTHS:
                    FOCAL_LENGTHS[class_name] = (pixel_width * KNOWN_DISTANCES[class_name]) / known_width
                
                # Calculate distance
                distance = calculate_distance(FOCAL_LENGTHS[class_name], known_width, pixel_width)
                
                # Calculate center point
                center = ((x1 + x2) // 2, (y1 + y2) // 2)
                
                # Update tracker and get motion analysis
                tracker.update(class_name, center, distance)
                motion, speed = tracker.get_motion_analysis(class_name)
                
                # Get collision prediction and stability analysis
                collision_status = predict_collision(center, tracker.object_history[class_name], frame.shape)
                stability = analyze_stability(tracker.object_history[class_name])
                
                # Update display info text
                info_text = f'{class_name}: {distance:.1f}in, {motion}, {stability}'
                
                # Display collision warning if needed
                if "Warning" in collision_status:
                    cv2.putText(frame, collision_status, (x1, y1-30), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            else:
                info_text = f'{class_name} ({confidence:.2f})'
            
            cv2.putText(frame, info_text, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.putText(frame, f'Press "q" to quit', (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.imshow('Household Object Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()